### Do preprocessing

In [13]:
import preprocess

train_df, dev_df, dev_baseline_df, test_df, evidence_df = preprocess.return_df()
original_train_df, original_dev_df, original_dev_baseline_df, original_test_df, original_evidence_df = preprocess.return_original_df()

In [14]:
# print head of the dataframes
print(train_df.head())
print(dev_df.head())
print(dev_baseline_df.head())
print(test_df.head())
print(evidence_df.head())
print("#####################################")
print(original_train_df.head())
print(original_dev_df.head())
print(original_dev_baseline_df.head())
print(original_test_df.head())
print(original_evidence_df.head())

                                                   claim_text  \
claim-1937  [scientific, evidence, co2, pollutant, higher,...   
claim-126   [el, nio, drove, record, high, global, tempera...   
claim-2510                 [1946, pdo, switched, cool, phase]   
claim-2021  [weather, channel, cofounder, john, coleman, p...   
claim-2449  [january, 2008, capped, 12, month, period, glo...   

                claim_label                                          evidences  
claim-1937         DISPUTED  [evidence-442946, evidence-1194317, evidence-1...  
claim-126           REFUTES                [evidence-338219, evidence-1127398]  
claim-2510         SUPPORTS                 [evidence-530063, evidence-984887]  
claim-2021         DISPUTED  [evidence-1177431, evidence-782448, evidence-5...  
claim-2449  NOT_ENOUGH_INFO  [evidence-1010750, evidence-91661, evidence-72...  
                                                   claim_text  \
claim-752   [south, australia, expensive, electricity, wor

### Doc2Vec encoding

In [ ]:
# Do doc2vec on the evidence_df
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pandas as pd

train_doc = train_df["claim_text"]
dev_doc = dev_df["claim_text"]
dev_baseline_doc = dev_baseline_df["claim_text"]
test_doc = test_df["claim_text"]
evidence_doc = evidence_df["value"]
tagged_data_raw = (
    # train_df["claim_text"].tolist() +
    # dev_df["claim_text"].tolist() +
    # dev_baseline_df["claim_text"].tolist() +
    evidence_df["value"].tolist()
)

print(tagged_data_raw[0:5])

tagged_data = [
    TaggedDocument(words=doc, tags=[str(i)])
    for i, doc in enumerate(tagged_data_raw)
    if isinstance(doc, list) and all(isinstance(w, str) for w in doc)
]

model = Doc2Vec(vector_size=20, min_count=2, epochs=100)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)


# Get vectors for claim_text
train_df["claim_vector"] = train_df["claim_text"].apply(
    lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
)

dev_df["claim_vector"] = dev_df["claim_text"].apply(
    lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
)

dev_baseline_df["claim_vector"] = dev_baseline_df["claim_text"].apply(
    lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
)

# Get vectors for evidence values
evidence_df["evidence_vector"] = evidence_df["value"].apply(
    lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
)

print(train_df.head())





[['scientific', 'evidence', 'co2', 'pollutant', 'higher', 'co2', 'concentration', 'actually', 'help', 'ecosystem', 'support', 'plant', 'animal', 'life'], ['el', 'nio', 'drove', 'record', 'high', 'global', 'temperature', 'suggesting', 'rise', 'may', 'manmade', 'emission'], ['1946', 'pdo', 'switched', 'cool', 'phase'], ['weather', 'channel', 'cofounder', 'john', 'coleman', 'provided', 'evidence', 'convincingly', 'refutes', 'concept', 'anthropogenic', 'global', 'warming'], ['january', '2008', 'capped', '12', 'month', 'period', 'global', 'temperature', 'drop', 'major', 'well', 'respected', 'indicator']]


[nltk_data] Downloading package punkt to C:\Users\Bill
[nltk_data]     Zhu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 